In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models

if __name__ == '__main__':

    # Device configuration
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def main():
        # Hyper-parameters
        batch_size = 100
        learning_rate = 1e-4
        num_classes = 10

        # Data augmentation and normalization for training
        transform_train = transforms.Compose([
            transforms.Resize(224),
            transforms.RandomHorizontalFlip(),
            transforms.RandomCrop(224, padding=4),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225]),
        ])

        transform_test = transforms.Compose([
            transforms.Resize(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225]),
        ])

        # CIFAR-10 dataset
        train_dataset = torchvision.datasets.CIFAR10(root='./data',
                                                    train=True,
                                                    transform=transform_train,
                                                    download=True)
        test_dataset = torchvision.datasets.CIFAR10(root='./data',
                                                    train=False,
                                                    transform=transform_test,
                                                    download=True)

        train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                                batch_size=batch_size,
                                                shuffle=True,
                                                num_workers=2)
        test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                                batch_size=batch_size,
                                                shuffle=False,
                                                num_workers=2)

        # Load the pretrained VGG16 model using the updated weights parameter
        model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)

        # Freeze feature extractor layers
        for param in model.features.parameters():
            param.requires_grad = False

        # Modify the classifier for CIFAR-10 (10 classes)
        model.classifier[6] = nn.Linear(in_features=4096, out_features=num_classes)
        model = model.to(device)

        # Loss and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.classifier.parameters(), lr=learning_rate)

        # Training loop
        total_step = len(train_loader)
        print(total_step)
        for epoch in range(5):
            model.train()
            running_loss = 0.0
            for i, (images, labels) in enumerate(train_loader):
                images = images.to(device)
                labels = labels.to(device)

                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)

                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()


                running_loss += loss.item()
                if (i+1) % 50 == 0:
                    print(f"Epoch [{epoch+1}/{5}], Step [{i+1}/{total_step}], Loss: {running_loss/100:.4f}")
                    running_loss = 0.0

        # Testing loop
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in test_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f"Test Accuracy of the model on the 10000 test images: {100 * correct / total:.2f}%")

    main()

100%|██████████| 170M/170M [00:18<00:00, 9.09MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:04<00:00, 124MB/s]


500
Epoch [1/5], Step [50/500], Loss: 0.4871
Epoch [1/5], Step [100/500], Loss: 0.2999
Epoch [1/5], Step [150/500], Loss: 0.2729
Epoch [1/5], Step [200/500], Loss: 0.2592
Epoch [1/5], Step [250/500], Loss: 0.2506
Epoch [1/5], Step [300/500], Loss: 0.2344
Epoch [1/5], Step [350/500], Loss: 0.2110
Epoch [1/5], Step [400/500], Loss: 0.2218
Epoch [1/5], Step [450/500], Loss: 0.2153
Epoch [1/5], Step [500/500], Loss: 0.2085
Epoch [2/5], Step [50/500], Loss: 0.1803
Epoch [2/5], Step [100/500], Loss: 0.1756
Epoch [2/5], Step [150/500], Loss: 0.1793
Epoch [2/5], Step [200/500], Loss: 0.1714
Epoch [2/5], Step [250/500], Loss: 0.1781
Epoch [2/5], Step [300/500], Loss: 0.1679
Epoch [2/5], Step [350/500], Loss: 0.1672
Epoch [2/5], Step [400/500], Loss: 0.1650
Epoch [2/5], Step [450/500], Loss: 0.1696
Epoch [2/5], Step [500/500], Loss: 0.1783
Epoch [3/5], Step [50/500], Loss: 0.1404
Epoch [3/5], Step [100/500], Loss: 0.1377
Epoch [3/5], Step [150/500], Loss: 0.1457
Epoch [3/5], Step [200/500], Loss